In [0]:
%fs
ls "/mnt/shop"

path,name,size,modificationTime
dbfs:/mnt/shop/gold-layer/,gold-layer/,0,1727800356000
dbfs:/mnt/shop/raw-data/,raw-data/,0,1727792291000
dbfs:/mnt/shop/silver-layer/,silver-layer/,0,1727799947000


In [0]:
Customer = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/shop/silver-layer/Customer.csv")
Generations = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/shop/silver-layer/Generations.csv")
Inventory = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/shop/silver-layer/Inventory.csv")
Product = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/shop/silver-layer/Product.csv")
SalesOrders = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/shop/silver-layer/SalesOrders.csv")
SalesOutlet = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/shop/silver-layer/SalesOutlet.csv")
SalesTarget = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/shop/silver-layer/SalesTarget.csv")
Staff = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/shop/silver-layer/staff.csv")

In [0]:
Customer.createOrReplaceTempView("Customer")
Generations.createOrReplaceTempView("Generations")
Inventory.createOrReplaceTempView("Inventory")
Product.createOrReplaceTempView("Product")
SalesOrders.createOrReplaceTempView("SalesOrders")
SalesOutlet.createOrReplaceTempView("SalesOutlet")
SalesTarget.createOrReplaceTempView("SalesTarget")
Staff.createOrReplaceTempView("Staff")

In [0]:
Customer_Dim = spark.sql("""SELECT 
                                c.customer_id, c.home_store, c.First_Name as first_name, c.customer_since, c.gender, c.birth_year,g.generation
                            FROM 
                                Customer c JOIN Generations g
                            ON c.birth_year = g.birth_year """)

Customer_Dim.show()

+-----------+----------+------------------+--------------+------+----------+------------+
|customer_id|home_store|        first_name|customer_since|gender|birth_year|  generation|
+-----------+----------+------------------+--------------+------+----------+------------+
|          1|         3|         Kelly Key|    2017-01-04|     M|      1950|Baby Boomers|
|          2|         3|   Clark Schroeder|    2017-01-07|     M|      1950|Baby Boomers|
|          3|         3|    Elvis Cardenas|    2017-01-10|     M|      1950|Baby Boomers|
|          4|         3|      Rafael Estes|    2017-01-13|     M|      1950|Baby Boomers|
|          5|         3|        Colin Lynn|    2017-01-15|     M|      1951|Baby Boomers|
|          6|         3|        Igor Beach|    2017-01-18|     M|      1951|Baby Boomers|
|          7|         3|      Scott Holden|    2017-01-21|     M|      1951|Baby Boomers|
|          8|         3|      Keegan Ayala|    2017-01-24|     M|      1951|Baby Boomers|
|         

In [0]:
Sales_fact = spark.sql("""
SELECT  
    transaction_id, transaction_date, outlet_id, customer_id, staff_id,
    product_id, quantity, unit_price
FROM SalesOrders 
""")

Sales_fact.show()


+--------------+----------------+---------+-----------+--------+----------+--------+-----------------+----------+
|transaction_id|transaction_date|outlet_id|customer_id|staff_id|product_id|quantity|       unit_price|instore_yn|
+--------------+----------------+---------+-----------+--------+----------+--------+-----------------+----------+
|        2069.0|      2019-04-22|      3.0|      247.0|    20.0|      28.0|     1.0|              2.0|     false|
|        1886.0|      2019-04-06|      5.0|     5879.0|    26.0|      33.0|     2.0|              3.5|      true|
|         590.0|      2019-04-22|      8.0|        0.0|    44.0|      45.0|     2.0|              3.0|      true|
|        1392.0|      2019-04-25|      8.0|        0.0|    44.0|      25.0|     1.0|2.200000047683716|      true|
|        1139.0|      2019-04-03|      5.0|     5126.0|    12.0|      47.0|     2.0|              3.0|      true|
|         968.0|      2019-04-26|      3.0|        0.0|    20.0|      50.0|     1.0|    

In [0]:
SalesOutlet_Dim = spark.sql("""
    SELECT 
        so.Outlet_ID, so.store_city, so.store_longitude, so.store_latitude, so.manager,
        st.beans_goal, st.beverage_goal, st.food_goal, st.merchandise_goal, st.total_goal
    FROM 
        SalesOutlet so JOIN SalesTarget st 
        ON so.Outlet_ID = st.Outlet_ID
    WHERE Outlet_Type LIKE('retail')
""")
SalesOutlet_Dim.show()

+---------+----------------+------------------+------------------+-------+----------+-------------+---------+----------------+----------+
|Outlet_ID|      store_city|   store_longitude|    store_latitude|manager|beans_goal|beverage_goal|food_goal|merchandise_goal|total_goal|
+---------+----------------+------------------+------------------+-------+----------+-------------+---------+----------------+----------+
|        3|Long Island City|-73.92401123046875| 40.76119613647461|    6.0|       720|        13500|     3420|             360|     18000|
|        4|        Brooklyn| -73.9839859008789| 40.67764663696289|   11.0|       720|        13500|     3420|             360|     18000|
|        5|        New York| -74.0101318359375| 40.71329116821289|   16.0|      1000|        18750|     4750|             500|     25000|
|        6|        New York|-73.99268341064453| 40.71385192871094|   21.0|       720|        13500|     3420|             360|     18000|
|        7|        New York|-73.95

In [0]:
Dates_Dim = spark.sql("""
SELECT DISTINCT
transaction_date, DATE_FORMAT(transaction_date, 'E') AS day_name,
DAY(transaction_date) AS day,
CASE
    WHEN DAY(transaction_date) <= 7 THEN 1
    WHEN DAY(transaction_date) <= 14 THEN 2
    WHEN DAY(transaction_date) <= 21 THEN 3
    ELSE 4
END AS week,
CASE 
    WHEN DAY(transaction_date) <= 10 THEN 1
    WHEN DAY(transaction_date) <= 20 THEN 2
    ELSE 3
END AS third
FROM SalesOrders
where transaction_date is not NULL
ORDER BY transaction_date
""")

Dates_Dim.show()

+----------------+--------+---+----+-----+
|transaction_date|day_name|day|week|third|
+----------------+--------+---+----+-----+
|      2019-04-01|     Mon|  1|   1|    1|
|      2019-04-02|     Tue|  2|   1|    1|
|      2019-04-03|     Wed|  3|   1|    1|
|      2019-04-04|     Thu|  4|   1|    1|
|      2019-04-05|     Fri|  5|   1|    1|
|      2019-04-06|     Sat|  6|   1|    1|
|      2019-04-07|     Sun|  7|   1|    1|
|      2019-04-08|     Mon|  8|   2|    1|
|      2019-04-09|     Tue|  9|   2|    1|
|      2019-04-10|     Wed| 10|   2|    1|
|      2019-04-11|     Thu| 11|   2|    2|
|      2019-04-12|     Fri| 12|   2|    2|
|      2019-04-13|     Sat| 13|   2|    2|
|      2019-04-14|     Sun| 14|   2|    2|
|      2019-04-15|     Mon| 15|   3|    2|
|      2019-04-16|     Tue| 16|   3|    2|
|      2019-04-17|     Wed| 17|   3|    2|
|      2019-04-18|     Thu| 18|   3|    2|
|      2019-04-19|     Fri| 19|   3|    2|
|      2019-04-20|     Sat| 20|   3|    2|
+----------

In [0]:
Staff_Dim = spark.sql("""
SELECT 
    staff_id, first_name, position, start_date as work_since, location
FROM Staff  
""")

Staff_Dim.show()

+--------+----------+---------------+----------+--------+
|staff_id|first_name|       position|work_since|location|
+--------+----------+---------------+----------+--------+
|       1|       Sue|            CFO|2001-08-03|      HQ|
|       2|       Ian|            CEO|2001-08-03|      HQ|
|       3|     Marny|        Roaster|2007-10-24|      WH|
|       4|   Chelsea|        Roaster|2003-07-03|      WH|
|       5|      Alec|        Roaster|2008-04-02|      WH|
|       6|      Xena|  Store Manager|2016-07-24|       3|
|       7|    Kelsey|Coffee Wrangler|2003-10-18|       3|
|       8|  Hamilton|Coffee Wrangler|2005-02-09|       3|
|       9|  Caldwell|Coffee Wrangler|2013-09-09|       3|
|      10|       Ima|Coffee Wrangler|2016-12-10|       3|
|      11|      Ruth|  Store Manager|2009-06-17|       4|
|      12|  Britanni|Coffee Wrangler|2006-03-25|       4|
|      13|      Berk|Coffee Wrangler|2009-12-11|       4|
|      14|     Damon|Coffee Wrangler|2010-06-05|       4|
|      15|  Re

In [0]:
Product_Dim = spark.sql("""
SELECT 
    product_id,category, sub_category,product_type,
    wholesale_price, retail_price, product_weight
FROM Product
""")

Product_Dim.show()

+----------+---------------+------------------+------------------+------------------+------------+--------------+
|product_id|       category|      sub_category|      product_type|   wholesale_price|retail_price|product_weight|
+----------+---------------+------------------+------------------+------------------+------------+--------------+
|         1|Whole Bean/Teas|      Coffee beans|     Organic Beans|14.399999618530273|        18.0|         12 oz|
|         2|Whole Bean/Teas|      Coffee beans| House blend Beans|14.399999618530273|        18.0|         12 oz|
|         3|Whole Bean/Teas|      Coffee beans|    Espresso Beans|11.800000190734863|       14.75|          1 lb|
|         4|Whole Bean/Teas|      Coffee beans|    Espresso Beans|16.360000610351562|       20.45|          1 lb|
|         5|Whole Bean/Teas|      Coffee beans|     Gourmet Beans|              12.0|        15.0|          1 lb|
|         6|Whole Bean/Teas|      Coffee beans|     Gourmet Beans|16.799999237060547|   

In [0]:
Product_Dim.toPandas().to_csv("/dbfs/mnt/shop/gold-layer/Product_Dim.csv", index = False)
Staff_Dim.toPandas().to_csv("/dbfs/mnt/shop/gold-layer/Staff_Dim.csv", index = False)
SalesOutlet_Dim.toPandas().to_csv("/dbfs/mnt/shop/gold-layer/SalesOutlet_Dim.csv", index = False)
Sales_fact.toPandas().to_csv("/dbfs/mnt/shop/gold-layer/Sales_fact.csv", index = False)
Customer_Dim.toPandas().to_csv("/dbfs/mnt/shop/gold-layer/Customer_Dim.csv", index = False)
Dates_Dim.toPandas().to_csv("/dbfs/mnt/shop/gold-layer/Dates_Dim.csv", index = False)